In [1]:
import gc
import glob
import pandas
import numpy as np

from datetime import datetime
from collections import defaultdict, Counter

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass

In [2]:
# ----- Декораторы -----

In [3]:
def scoreTime(name):
    '''Декоратор для вывода времени работы функции (начало-конец)'''
    def wrapper(func):
        def function(*args, **kwargs):
            print('\r', name, ': start\t', datetime.now())
            res = func(*args, **kwargs)
            print('\r', name, ': end  \t', datetime.now())
            return res
        return function
    return wrapper

In [4]:
# ----- Функции работы с файлами: removeFiles, save -----

In [5]:
@scoreTime('removeFiles')
def removeFiles(files, log=False):
    ''' Функция удаления файлов '''
    chunks = len(files)
    percent = -1
    for file in files:
        if (log) and (round(i * 100 / chunks) != percent):
            percent = round(i * 100 / chunks)
            print('\r', f'{percent}% chunk: {i+1}/{chunks+1}', end='   ')
        os.remove(file)
    print('\r', f'100% chunk: {chunks+1}/{chunks+1}')
# ----------------------------------------------------------------
def save(df, path, index = False, format = 'parquet'):
    func = df.to_csv if format == 'csv' else df.to_parquet
    func(f'{path}.{format}', index=index)

In [6]:
# ----- Функции разделения датасета на файлы -----

In [7]:
@scoreTime('trimFiles')
def trimFiles(df, time_elapsed, saveFunc, log=False):
    ''' Разбиваем данные на временные промежутки
        и считаем пересечения внутри них '''
    df = df.sort_values(by=['ts'])
    min_ts, max_ts = df.iloc[0]['ts'], df.iloc[-1]['ts']
    chunks = round((max_ts - min_ts) / time_elapsed)
    percent = -1
    for i in range(0, chunks):
        if (log) and (round(i * 100 / chunks) != percent):
            percent = round(i * 100 / chunks)
            print('\r', f'{percent}% chunk: {i+1}/{chunks+1}', end='   ')
        start = min_ts + i * time_elapsed
        df_chank = df[(df['ts'] >= start)&(df['ts'] <= start + time_elapsed)].drop('ts', axis=1)
        if df_chank.shape[0] > 1:
            saveFunc(df_chank, f'chunk_{i}')
        gc.collect()
    print('\r', f'100% chunk: {chunks+1}/{chunks+1}')

In [8]:
# -----------------------------------------
def mergeSimpleDF(df, path, time_elapsed = None):
    df = df.merge(df, on='session')
    df = df[['session', 'aid_x', 'aid_y']]
    df = df[df['aid_x'] != df['aid_y']]
    df = df.drop_duplicates(['session', 'aid_x', 'aid_y'])
    df = df[['aid_x', 'aid_y']]
    df['wgt'] = 1
    df = df.groupby(['aid_x', 'aid_y']).wgt.sum().reset_index()
    df[['aid_x', 'aid_y']] = df[['aid_x', 'aid_y']].astype('uint32')
    df['wgt'] = df['wgt'].astype('uint16')
    df.sort_values('aid_x').to_parquet(f'{path}.parquet')

In [9]:
# ----- Функции расчёта весов -----

In [10]:
def mostCommon(files, count=20, filename='co_matrix'):
    chunks = len(files)
    percent = -1
    matrix = pd.DataFrame()
    for i, fleft in enumerate(files):
        if (round(i * 100 / chunks) != percent):
            percent = round(i * 100 / chunks)
            print('\r', f'{percent}% chunk: {i + 1}/{chunks + 1}', end='   ')
        left  = pd.read_parquet(fleft)
        left = left.sort_values(['aid_x', 'wgt'], ascending=False).reset_index(drop=True)
        left['n'] = left.groupby('aid_x').cumcount()
        left = left[left['n'] < count].drop('n',axis=1)
        matrix = pd.concat([matrix, left])
        matrix = matrix.sort_values(['aid_x', 'wgt'], ascending=False).reset_index(drop=True)
        matrix['n'] = matrix.groupby('aid_x').cumcount()
        matrix = matrix[matrix['n'] < count].drop('n',axis=1)
        del left
        gc.collect()
    print('\r', f'100% chunk: {chunks+1}/{chunks+1}')
    return matrix.drop('wgt', axis=1)

In [11]:
@scoreTime('calcWGT')
def calcWGT(files):
    chunks = len(files) * len(files)
    percent = -1
    for i, fleft in enumerate(files):
        left  = pd.read_parquet(fleft)
        for j, fright in enumerate(files[i+1:]):
            if (round((i * len(files) + j) * 100 / chunks) != percent):
                percent = round((i * len(files) + j) * 100 / chunks)
                print('\r', f'{percent}% file: {(i * len(files) + j) + 1}/{chunks + 1}', end='   ')
            right = pd.read_parquet(fright)
            left = left.merge(right, how='left', on=['aid_x', 'aid_y']).fillna(0)
            left['wgt_x'] += left['wgt_y']
            left = left.drop(['wgt_y'], axis=1).sort_values('aid_x').rename(columns={'wgt_x':'wgt'})
            duplicates = pd.concat([left, right], ignore_index=True)\
                            .drop_duplicates(subset=['aid_x', 'aid_y'])
            right = duplicates[duplicates.index >= left.shape[0]][['aid_x', 'aid_y', 'wgt']]
            right.to_parquet(fright ,index=False)
            del duplicates, right
            gc.collect()
        left.to_parquet(fleft ,index=False)
        gc.collect()
    print('\r', f'100% chunk: {chunks+1}/{chunks+1}')
# -----------------------------------------------------------------------
@scoreTime('calcWGTWeighted')
def calcWGTWeighted(files):
    chunks = len(files) * len(files)
    percent = -1
    for i, fleft in enumerate(files):
        left  = pd.read_parquet(fleft)
        for j, fright in enumerate(files[i+1:]):
            if (round((i * len(files) + j) * 100 / chunks) != percent):
                percent = round((i * len(files) + j) * 100 / chunks)
                print('\r', f'{percent}% file: {(i * len(files) + j) + 1}/{chunks + 1}', end='   ')
            right = pd.read_parquet(fright)
            left = left.merge(right, how='left', on=['aid_x', 'aid_y']).fillna(0)
            left['wgt_x'] += left['wgt_y']
            left = left.drop(['wgt_y'], axis=1).sort_values('aid_x').rename(columns={'wgt_x':'wgt'})
            duplicates = pd.concat([left, right], ignore_index=True)\
                            .drop_duplicates(subset=['aid_x', 'aid_y'])
            right = duplicates[duplicates.index >= left.shape[0]][['aid_x', 'aid_y', 'wgt']]
            right.to_parquet(fright ,index=False)
            del duplicates, right
            gc.collect()
        left.to_parquet(fleft ,index=False)
        gc.collect()
    print('\r', f'100% chunk: {chunks+1}/{chunks+1}')

In [12]:
# ----- Пайплайны расчёта матрицы -----

In [13]:
def withTrimFiles(path, wgtFunc, drop=[]):
    df = pd.read_parquet(path)
    if len(drop) > 0: df.drop(drop, axis=1, inplace=True)
    trimFiles(df, params['time_elapsed'], mergeSimpleDF, log=True)
    del df; gc.collect()
    wgtFunc(glob.glob('./chunk_*.parquet'))
    df = mostCommon(glob.glob('./chunk_*.parquet'))
    removeFiles(glob.glob('./chunk_*.parquet'))
    if GPU: df = df.to_pandas()
    df = df.groupby('aid_x').agg(lambda x: list(dict.fromkeys(x)))
    if GPU: df = pd.from_pandas(df)
    return df

In [14]:
# ==================================================================
# ----- Конфигурация сессии для экспериментов-----------------------
# ==================================================================
#LOCAL = True  # Тип метрики. True - локально, False - Kaggle
SAVE  = True # Сохранять ли файлы. Нужно при сохранении датасета
SHOW  = True # Выводить ли таблицы в процессе
GPU   = True # Использовать GPU
# -----------------------------------------------------------------
params = {
    'path': '/otto-analyse-data/train_no_duplicates.parquet',
    'drop': [],
    'time_elapsed': 24*60*60,
    'func': withTrimFiles,
    'wgtFunc': calcWGT,
}
# -----------------------------------------------------------------
if GPU: import cudf
pd = cudf if GPU else pandas
print('We will use RAPIDS version',cudf.__version__) if GPU else print('We will use CPU...')

We will use RAPIDS version 21.10.01


In [15]:
# ----- Обработка файлов -----

In [16]:
%%time
# ~ 17m (GPU)
matrix = params['func']('../input' + params['path'], params['wgtFunc'], params['drop'])
if SHOW: display(matrix)
if SAVE: 
    tmp = matrix.reset_index()
    tmp.columns = ['aid', 'labels']
    save(tmp, 'covisit_matrix')
    del tmp; gc.collect()

 trimFiles : start	 2023-01-09 15:31:07.416111
 100% chunk: 36/36
 trimFiles : end  	 2023-01-09 15:32:43.957996
 calcWGT : start	 2023-01-09 15:32:44.120327
 100% chunk: 1226/1226
 calcWGT : end  	 2023-01-09 15:50:27.891023
 100% chunk: 36/36
 removeFiles : start	 2023-01-09 15:51:19.455485
 100% chunk: 36/36
 removeFiles : end  	 2023-01-09 15:51:20.237099


,aid_y
aid_x,
0,"[532042, 643097, 1848174, 1735605, 1350484, 70..."
1,"[1533875, 28092, 166683, 1234826, 990115, 1904..."
2,"[881881, 1101201, 1811594, 490493, 929227, 180..."
3,"[1180285, 170046, 1771163, 776187, 1512013, 83..."
4,"[536431, 1823804, 485093, 207360, 1805675, 641..."
...,...
1855598,"[76867, 1611306, 38444, 213351, 336719, 184371..."
1855599,"[315380, 1674841, 1638856, 1705997, 405852, 99..."
1855600,"[700554, 1136772, 69211, 619488, 1801680, 1807..."


CPU times: user 8min 58s, sys: 6min 29s, total: 15min 27s
Wall time: 20min 58s


In [17]:
# ----- PREDICT -----

In [18]:
def groupType(x):
    res = list(set(x))
    return sum(res) if len(res) > 0 else -1
# -----------------------------------------------------------
def groupLen(x):
    return len(list(set(x)))
# -----------------------------------------------------------
def fillEvents(df, clone=True, value=[-1]):
    import pandas as pnd
    result = pnd.DataFrame()
    if GPU: df = df.to_pandas()
    _ = df[['session','type']].groupby(['session'])\
            .agg(ctype = ('type', groupType), count = ('type', groupLen))
    
    tmp = df[df['session'].isin(_[_['ctype'] == 0].index)].copy()
    if not clone: tmp['aid'] = tmp['aid'].apply(lambda x: value)
    tmp['type'] = 1
    result = pnd.concat([result, tmp])
    tmp['type'] = 2
    result = pnd.concat([result, tmp])

    tmp = df[df['session'].isin(_[_['ctype'] == 1].index)].copy().drop_duplicates(['session'])
    if not clone: tmp['aid'] = tmp['aid'].apply(lambda x: value)
    tmp['type'] = 2
    result = pnd.concat([result, tmp])

    tmp = df[df['session'].isin(_[_['ctype'] == 2].index)].copy().drop_duplicates(['session'])
    if not clone: tmp['aid'] = tmp['aid'].apply(lambda x: value)
    tmp['type'] = 1
    result = pnd.concat([result, tmp])
    
    tmp = df[df['session'].isin(_[(_['ctype'].isin([1, 2]))&(_['count'] == 1)].index)].copy()
    if not clone: tmp['aid'] = tmp['aid'].apply(lambda x: value)
    tmp['type'] = 0
    result = pnd.concat([result, tmp])

    tmp = df[df['session'].isin(_[(_['ctype'] == 3)&(_['count'] == 2)].index)].copy()\
                .drop_duplicates(['session'])
    if not clone: tmp['aid'].apply(lambda x: [])
    tmp['type'] = 0
    result = pnd.concat([result, tmp])
    if GPU: result = pd.from_pandas(result)
    if GPU: df = pd.from_pandas(df)
    return pd.concat([df, result])

In [19]:
coMatrix = matrix.copy().reset_index()
coMatrix.columns = ['aid', 'labels']
if SHOW: display(coMatrix)

,aid,labels
0,0,"[532042, 643097, 1848174, 1735605, 1350484, 70..."
1,1,"[1533875, 28092, 166683, 1234826, 990115, 1904..."
2,2,"[881881, 1101201, 1811594, 490493, 929227, 180..."
3,3,"[1180285, 170046, 1771163, 776187, 1512013, 83..."
4,4,"[536431, 1823804, 485093, 207360, 1805675, 641..."
...,...,...
1852312,1855598,"[76867, 1611306, 38444, 213351, 336719, 184371..."
1852313,1855599,"[315380, 1674841, 1638856, 1705997, 405852, 99..."
1852314,1855600,"[700554, 1136772, 69211, 619488, 1801680, 1807..."
1852315,1855601,"[247249, 1624652, 1345252, 376752, 656860, 148..."


In [20]:
%%time
# ~ 7s 
test = pd.read_parquet('../input/otto-analyse-data/test.parquet').drop('ts', axis=1)
test = fillEvents(test)
if SHOW: display(test)

,session,aid,type
0,12899779,59625,0
1,12899780,1142000,0
2,12899780,582732,0
3,12899780,973453,0
4,12899780,736515,0
...,...,...,...
6374308,14417710,803722,0
6419994,14429044,688179,0
6434531,14432851,1807937,0
6538408,14459954,180271,0


CPU times: user 23.6 s, sys: 640 ms, total: 24.2 s
Wall time: 25 s


In [21]:
def mergeCoMatrix(df):
    return pd.merge(df, coMatrix, how='left', on=['aid'])

In [22]:
test = mergeCoMatrix(test)
if SHOW: display(test)

,session,aid,type,labels
0,12900392,997639,0,"[30361, 1115393, 409467, 637119, 266589, 11752..."
1,12900392,997639,0,"[30361, 1115393, 409467, 637119, 266589, 11752..."
2,12900392,997639,0,"[30361, 1115393, 409467, 637119, 266589, 11752..."
3,12900392,997639,0,"[30361, 1115393, 409467, 637119, 266589, 11752..."
4,12900392,439178,0,"[315448, 671451, 1218027, 361830, 1420819, 365..."
...,...,...,...,...
15242866,14214491,1043508,0,"[634452, 1734305, 1079588, 986164, 224345, 727..."
15242867,14192387,361549,0,"[69400, 23428, 733994, 1540277, 246221, 174987..."
15242868,14204524,923599,0,"[1264313, 1624436, 554028, 1252357, 178845, 63..."
15242869,14198134,546755,0,"[1796103, 1680387, 674590, 1374175, 199409, 22..."


In [23]:
from math import ceil
lastSize = 20
# -------------------------------------------------
def getLastN(arr, size = 20):
    count = ceil(size / len(arr))
    max_len = 0
    result = []
    start = 0
    free = []
    while count > 0 and max_len < size:
        next_for_all = 0
        for i, row in enumerate(arr):
            if type(row) == float: continue
            if i in free: continue
            if max_len >= size: break
            if len(row) < start + count:
                next_for_all += start + count - len(row)
                result.extend(row[start:])
                max_len += len(row[start:])
                free.append(i)
            else:
                result.extend(row[start:start + count])
                max_len += count
        clen = len(arr) - len(free)
        if clen == 0:
            break
        start += count
        count = ceil(next_for_all / clen)
    return result
# -------------------------------------------------
def aggSet(x):
    return list(dict.fromkeys(x))
# -------------------------------------------------
def aggLabel(x):
    global lastSize
    return getLastN(list(x), lastSize)
# -----------------------------------------------------------------------
def getPreds(df, size = 20):
    '''Получаем предсказание с учётом матрицы вероятностей '''
    global lastSize
    lastSize = size
    if GPU: df = df.to_pandas()
    df['labels'].fillna('', inplace=True)
    result = df.groupby(['session', 'type']).agg(labels = ('labels', aggLabel))
    if GPU: result = pd.from_pandas(result)
    result.reset_index(inplace=True)
    return result

In [24]:
def dropAidDuplicates(df, column = 'labels', count=20):
    tmp = df.copy()
    if GPU: tmp = tmp.to_pandas()
    tmp[column] = tmp[column].map(lambda x: list(dict.fromkeys(x))[:count])
    return pd.from_pandas(tmp) if GPU else tmp

In [25]:
%%time
# - 1m 30s
res_preds = getPreds(test, 20)
if SHOW: display(res_preds)

,session,type,labels
0,12899779,0,"[737445, 731692, 941596, 438191, 1253524, 1340..."
1,12899779,1,"[737445, 731692, 941596, 438191, 1253524, 1340..."
2,12899779,2,"[737445, 731692, 941596, 438191, 1253524, 1340..."
3,12899780,0,"[1502122, 487136, 889686, 636101, 1758603, 114..."
4,12899780,1,"[1502122, 487136, 889686, 636101, 1758603, 114..."
...,...,...,...
5015404,14571580,1,"[1314576, 433425, 871658, 925638, 1231403, 682..."
5015405,14571580,2,"[1314576, 433425, 871658, 925638, 1231403, 682..."
5015406,14571581,0,"[1684953, 462056, 1158237, 622489, 1401429, 11..."
5015407,14571581,1,"[1684953, 462056, 1158237, 622489, 1401429, 11..."


CPU times: user 1min 47s, sys: 10.3 s, total: 1min 57s
Wall time: 1min 58s


In [26]:
%%time
# ~s
res_preds = dropAidDuplicates(res_preds, 'labels')
res_preds

CPU times: user 41.9 s, sys: 4.76 s, total: 46.7 s
Wall time: 46.4 s


,session,type,labels
0,12899779,0,"[737445, 731692, 941596, 438191, 1253524, 1340..."
1,12899779,1,"[737445, 731692, 941596, 438191, 1253524, 1340..."
2,12899779,2,"[737445, 731692, 941596, 438191, 1253524, 1340..."
3,12899780,0,"[1502122, 487136, 889686, 636101, 1758603, 114..."
4,12899780,1,"[1502122, 487136, 889686, 636101, 1758603, 114..."
...,...,...,...
5015404,14571580,1,"[1314576, 433425, 871658, 925638, 1231403, 682..."
5015405,14571580,2,"[1314576, 433425, 871658, 925638, 1231403, 682..."
5015406,14571581,0,"[1684953, 462056, 1158237, 622489, 1401429, 11..."
5015407,14571581,1,"[1684953, 462056, 1158237, 622489, 1401429, 11..."


In [27]:
# ----- SUBMISSION -----

In [28]:
def getSubmission(df):
    sub = df.copy().dropna()
    if GPU: sub = sub.to_pandas()
    sub['labels'] = sub['labels'].map(lambda x: ' '.join(str(i) for i in list(dict.fromkeys(x))))
    sub['type'] = sub['type'].map({0: 'clicks', 1: 'carts', 2: 'orders'})
    sub['session_type'] = sub['session'].astype('str') + '_' + sub['type']
    sub.drop(['session', 'type'], axis=1, inplace=True)
    return pd.from_pandas(sub) if GPU else sub

In [29]:
%%time
# ~s
submission = getSubmission(res_preds)
if SAVE: submission.to_csv('submission.csv', index=False)
if SHOW: display(submission)

,labels,session_type
0,737445 731692 941596 438191 1253524 1340695 17...,12899779_clicks
1,737445 731692 941596 438191 1253524 1340695 17...,12899779_carts
2,737445 731692 941596 438191 1253524 1340695 17...,12899779_orders
3,1502122 487136 889686 636101 1758603 1142000 7...,12899780_clicks
4,1502122 487136 889686 636101 1758603 1142000 7...,12899780_carts
...,...,...
5015404,1314576 433425 871658 925638 1231403 682237 88...,14571580_carts
5015405,1314576 433425 871658 925638 1231403 682237 88...,14571580_orders
5015406,1684953 462056 1158237 622489 1401429 1124107 ...,14571581_clicks
5015407,1684953 462056 1158237 622489 1401429 1124107 ...,14571581_carts


CPU times: user 1min 9s, sys: 4.48 s, total: 1min 13s
Wall time: 1min 13s
